<a href="https://colab.research.google.com/github/DellsadaLvt/Sample_Code/blob/New-branch/Chatbot_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#load file
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


In [ ]:
pip install tensorflow


In [ ]:
pip install tflearn

     |████████████████████████████████| 112kB 6.7MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=a837f4d960741dbc77764ec68a4a687ec85d82741608647d81312ba9c7d4e9fb
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [ ]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import json
import pickle

In [ ]:
import nltk
nltk.download('punkt')
from tensorflow.python.framework import ops
ops.reset_default_graph()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
 
import numpy
import tflearn
import tensorflow
import random
import json
 
 
with open("intents.json") as file:
    data = json.load(file)
 
 
 
words = []
labels = []
docs_x = []
docs_y = []
 
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
 
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
 
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
 
labels = sorted(labels)
 
training = []
output = []
 
out_empty = [0 for _ in range(len(labels))]
 
for x, doc in enumerate(docs_x):
    bag = []
 
    wrds = [stemmer.stem(w.lower()) for w in doc]
 
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
 
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
 
    training.append(bag)
    output.append(output_row)
 
 
training = numpy.array(training)
output = numpy.array(output)

In [ ]:
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [48]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 9599  | total loss: 0.00485 | time: 0.007s
| Adam | epoch: 1400 | loss: 0.00485 - acc: 1.0000 -- iter: 24/26
Training Step: 9600  | total loss: 0.00511 | time: 0.009s
| Adam | epoch: 1400 | loss: 0.00511 - acc: 1.0000 -- iter: 26/26
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [49]:
with open("intents.json") as file:
    data = json.load(file)
 
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
 
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
 
        if intent["tag"] not in labels:
            labels.append(intent["tag"])
 
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
 
    labels = sorted(labels)
 
    training = []
    output = []
 
    out_empty = [0 for _ in range(len(labels))]
 
    for x, doc in enumerate(docs_x):
        bag = []
 
        wrds = [stemmer.stem(w.lower()) for w in doc]
 
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
 
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
 
        training.append(bag)
        output.append(output_row)
 
 
    training = numpy.array(training)
    output = numpy.array(output)
 
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)
 
ops.reset_default_graph()
 
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)
 
model = tflearn.DNN(net)
 
try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [53]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        print(results)

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
[[2.5103230e-03 1.5419335e-03 9.9594742e-01 2.2627954e-07 4.7119926e-09
  1.7137191e-07]]
Good to see you again!
You: ada
[[0.04519845 0.18896446 0.75044364 0.00362969 0.00354317 0.00822062]]
Hi there, how can I help?
You: quit


In [ ]:
#### tesst

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        print(results_index)
    
        #if results_index > 
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        print(random.choice(responses))

        

chat()

Start talking with the bot (type quit to stop)!
You: hi
2
Hi there, how can I help?
You: menu
1
Talk to you later
You: the menu
5
Cookies are on the menu!
You: another
2
Good to see you again!
You: what on the menu
5
Cookies are on the menu!
You: quit
